In [1]:
import json
import pandas as pd
from sql_queries import ValorantQueries

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [4]:
states = vq.get_initial_state()
events = vq.get_event_table()
player_alive_dict = {item: True for item in states["Player ID"].unique()}

current_round_alive_dict = player_alive_dict.copy()
chosen_round = 2
current_round_states = states[states['Round'] == chosen_round]
current_round_events = events[events['Round'] == chosen_round]

attacking_players = tuple(current_round_states[current_round_states['Starting Side'] == 'attack']['Player ID'].unique()) if chosen_round <= 12 else tuple(current_round_states[current_round_states['Starting Side'] == 'defense']['Player ID'].unique())
defending_players = tuple(current_round_states[current_round_states['Starting Side'] == 'defense']['Player ID'].unique()) if chosen_round <= 12 else tuple(current_round_states[current_round_states['Starting Side'] == 'attack']['Player ID'].unique())

current_gamestate_list = []

for event in current_round_events.iterrows():
    event_type = event[1]["EventType"]
    victim_id = event[1]["VictimID"]
    if event_type == "kill":
        current_round_alive_dict[victim_id] = False
        gs = vq.get_current_gamestate(current_round_states, current_round_alive_dict, attacking_players, defending_players)
        current_gamestate_list.append(gs)
    player_data = current_round_states[current_round_states['Player ID'] == victim_id]

### AAA